# Entrega L2 - Aprendizaje Bayesiano

### Grupo 38:
     - S. Robaina  C.I... 5.310.579-8
     - A. Baptista C.I... 4.883.322-3


##  Introducción

Se busca probar el modelo algoritmo de clasificación de naive bayes, aplicando al contexto de clasificación de texto y para eso

## 1. Datos

El conjunto de datos de entrenamiento fue extraído de la aplicación de mensajería _WhatsApp_, y cuenta con  mensajes compuestos por un total de  palabras.

## 2. Objetivo

*   Implementar el algoritmo _Naive Bayes_ para desarrollar un modelo que permita predecir palabras.

*   Comparar su desempeño variando el hiperparámetro horizonte(N).

*   Evaluar su capacidad de aprendizaje realizando reentrenando el modelo.

## 3. Diseño

### 3.1 Preprocesamiento de datos
*   Como el formato en el que se exportan los mensajes de _WhatsApp_ es el siguiente:
    *   D/M/AA HH:MM - Nombre: (mensaje)
    
    Se extrajeron los mensajes conservando únicamente el contenido posterior a ":" en cada línea. Luego se generó una lista compuesta de las palabras de los mensajes conservados.

*   Al exportar los mensajes de _WhatsApp_ a un archivo de texto se incluyen algunas frases que no son mensajes como tal, por ejemplo: "Se eliminó este mensaje", "Multimedia omitido", etc. Evidenciamos que estas frases pueden tener un peso estadístico considerable, por lo que decidimos eliminarlas de los datos para no introducir un sesgo en el modelo generado.

*   Como el lenguaje en _WhatsApp_ no suele ser muy riguroso en términos de errores ortográficos, situación que introduce ruido a nuestros datos. Para resolver este problema probamos cruzar las palabras con un diccionario en español (disponible en: https://github.com/words/an-array-of-spanish-words), descartando aquellas palabras no presentes en el mismo. 
El problema es que este diccionario no incluye palabras con tilde, por lo que elimina un gran número de palabras que no contenían errores ortográficos. Para lidiar con esta situación nos planteamos dos posibilidades:
    *   Que las palabras que no se encontraran en el diccionario fueran corregidas utilizando la biblioteca _pyspellchecker_. Esta opción se descarta debido a que el tiempo de procesamiento era demasiado elevado.
    
    *   Eliminar los tildes de las palabras previo a cruzarlas con el diccionario. Reconocemos que esta alternativa nos genera una limitante importante ya que la acentuación es de vital importancia en el idioma español. De todas formas se probó esta alternativa y se la comparó a los resultados utilizando datos sin corrección ortográfica, con el fin de experimentar y evaluar el ruido introducido por errores ortográficos.

### 3.2 Evaluación
*   El desempeño de los distintos modelos generados se evaluó mediante el ingreso de 5 inicios de frases diferentes (por ejemplo: "buenas", "si yo opino que", etc.), permitiendo al modelo hacer hasta 5 sugerencias de palabras luego de ingresarlas. De las sugerencias realizadas por los modelos se evaluó: 

    *   Confiabilidad de la sugerencia: Se calcula el promedio de los factores de probabilidad de cada palabra sugerida. Esto permite que las métricas obtenidas para distintos valores de horizonte(N) sean más comparables, especialmente para valores de N más grandes. Para comparar el desempeño de la frase generada en su totalidad, se realizó un promedio de las confiabilidades de las palabras sugeridas. 

    *   Coherencia o sentido de la sugerencia: Se asignó un valor subjetivo del 1 al 5 que evaluará el nivel de coherencia de las frases generadas por los modelos al aceptar las 5 sugerencias. 

## 4. Experimentación

## 6. Evaluación de los modelos

* 

## 7. Conclusiones